In [1]:
# Paso 1: Autenticación y Descarga Automática de Datos desde Kaggle
from google.colab import files

# Sube el archivo kaggle.json aquí
files.upload()

# Configuración de Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargar y descomprimir datos de Kaggle
!kaggle competitions download -c u-tad-dogs-vs-cats-2024
!unzip -q u-tad-dogs-vs-cats-2024.zip -d /content/dogs-vs-cats

# Paso 2: Importar Bibliotecas
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import data as tf_data
import keras

# Paso 3: Configuración Inicial
seed = 1
keras.utils.set_random_seed(seed)

image_size = (600, 600)  # Aumentamos el tamaño de la imagen para EfficientNetB7
batch_size = 32  # Disminuimos el batch_size para mejorar el rendimiento del entrenamiento

# Paso 4: Cargar Datos
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/content/dogs-vs-cats/train/train",
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
)

# Paso 5: Configurar el Modelo
efficientnet = keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(600, 600, 3))
efficientnet.trainable = False  # Congelamos las capas preentrenadas inicialmente

from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D

# Añadimos capas de clasificación al final
x = efficientnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Añadimos una capa densa intermedia
new_classification_layer = Dense(2, activation='softmax')(x)
model_new = Model(efficientnet.input, new_classification_layer)

# Hacemos que las nuevas capas añadidas sean entrenables
for layer in efficientnet.layers:
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])

model_new.summary()

# Paso 6: Entrenamiento del Modelo
epochs = 25  # Aumentamos el número de épocas

history = model_new.fit(train_ds,
                        validation_data=val_ds,
                        epochs=epochs)

# Paso 7: Visualización de Resultados
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:, "loss"], lw=2, label='training loss')
plt.plot(logs.loc[1:, "val_loss"], lw=2, label='validation loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:, "accuracy"], lw=2, label='training accuracy')
plt.plot(logs.loc[1:, "val_accuracy"], lw=2, label='validation accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')
plt.show()

# Paso 8: Predicción
folder_path = "/content/dogs-vs-cats/test/test"
discrimination_threshold = 0.5  # below 0.5 is a cat, above 0.5 is a dog
predictions_dict = {}

for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)

    # save the image name
    file_name = img.split('/')[-1]
    file_no_extension = file_name.split('.')[0]

    img = keras.utils.load_img(img, target_size=image_size)
    img_array = keras.utils.img_to_array(img)
    img_array = keras.ops.expand_dims(img_array, 0)
    predictions = model_new.predict(img_array, verbose=None)
    score = predictions[0][1]
    label = (score > discrimination_threshold) * 1

    # save the predictions to a dictionary
    predictions_dict[int(file_no_extension)] = label

# Crear el archivo de envío
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"]).sort_values(by='id', ascending=True)
submission.to_csv('submission.csv', index=False)

# Mostrar números de cada clase etiqueta
submission["label"].value_counts()
submission


Saving kaggle.json to kaggle.json
100% 1.87G/1.87G [01:36<00:00, 21.5MB/s]
100% 1.87G/1.87G [01:36<00:00, 20.7MB/s]
Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Using 5000 files for validation.
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 600, 600, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 600, 600, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 600, 600, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 600, 600, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 601, 601, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 300, 300, 64)   │          1,728 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 300, 300, 64)   │            256 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 300, 300, 64)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 300, 300, 64)   │            576 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 300, 300, 64)   │            256 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 300, 300, 64)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 64)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 64)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 16)       │          1,040 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 64,753,817 (247.02 MB)

 Trainable params: 64,443,090 (245.83 MB)

 Non-trainable params: 310,727 (1.19 MB)

Epoch 1/25


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt